In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [65]:
import utils
word = 'вилка'
weights = utils.load_weights('../corpora/ad-nouns/cdict/', word)
texts = utils.load_contexts('../corpora/ad-nouns-contexts-100k', word)
texts = [[w for w in ctx if weights.get(w, 0) > 1] for ctx in texts]
texts = [ctx for ctx in texts if ctx]
for text in texts[:3]:
    print(text)
print(len(texts))
#texts = texts[:1000]

['помазок', 'картофелина', 'тряпочка', 'наматывать']
['нож']
['звон', 'нож']
20951


In [66]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[0], corpus[1])

[(0, 1), (1, 1), (2, 1), (3, 1)] [(4, 1)]


In [78]:
#from gensim.models.ldamulticore import LdaMulticore as LdaModel
from gensim.models import LdaModel
from gensim.models import HdpModel
print(len(corpus))
#lda = HdpModel(corpus, id2word=dictionary)
lda = LdaModel(corpus, id2word=dictionary, num_topics=6, passes=3, iterations=100, alpha='auto')

20951


In [87]:
lda.print_topics(num_words=5)
for topic_id in range(lda.num_topics):
    terms = lda.get_topic_terms(topic_id, topn=5)
    print(topic_id, ' '.join(dictionary[wid] for wid, _ in terms))
#lda.print_topics(topics=6, topn=5)

0 взять откладывать розетка палец предохранитель
1 передний колесо масло задний яйцо
2 розетка втыкать вынимать провод выдергивать
3 нож ложка тарелка стол прибор
4 рука держать выпивать огурец банка
5 нож рука рот кусок мясо


In [61]:
import rl_wsd_labeled
senses, contexts = rl_wsd_labeled.get_contexts(rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', word))
print(senses)
print(contexts[0])

{'2': 'Книга с репродукциями или фотографиями', '1': 'Тетрадь или книга с чистыми листами', '3': 'Собрание музыкальных произведений'}
(('достаточно лишь колесиком мышки крутить вниз. И если вы захотите увеличить фотографию в', ' альбоме', ', то все следующие фотографии будут также отображаться в полноразмерном варианте'), '2')


In [62]:
print(contexts[0][0])
print(utils.normalize(contexts[0][0]))

('достаточно лишь колесиком мышки крутить вниз. И если вы захотите увеличить фотографию в', ' альбоме', ', то все следующие фотографии будут также отображаться в полноразмерном варианте')
['достаточно', 'лишь', 'колесико', 'мышка', 'крутить', 'вниз', 'захотеть', 'увеличивать', 'фотография', 'следующий', 'фотография', 'также', 'отображаться', 'полноразмерный', 'вариант']


In [80]:
import numpy as np
documents = [dictionary.doc2bow(utils.normalize(ctx)) for ctx, _ in contexts]
gamma, _ = lda.inference(documents)
pred_topics = gamma.argmax(axis=1)
true_labels = np.array([int(ans) for _, ans in contexts])
print(pred_topics[:10])
print(true_labels[:10])

[3 3 3 3 3 3 3 0 3 3]
[2 3 2 3 3 3 2 3 2 3]


In [81]:
from sklearn.metrics import v_measure_score, adjusted_rand_score
print('ARI:', adjusted_rand_score(true_labels, pred_topics))
print('V score:', v_measure_score(true_labels, pred_topics))

ARI: 0.0911742420850017
V score: 0.0321246695993
